In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define the path to the uploaded zip file
zip_path = "/content/drive/MyDrive/archive.zip"
extract_path = "/content/drive/MyDrive"  # Destination folder

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the extracted files
extracted_files = os.listdir(extract_path)
print("Extracted Files:", extracted_files)


Extracted Files: ['archive.zip', 'finetune_stable_diffusion', 'Copy of Untitled2 (1).ipynb', 'major_project.ipynb', 'Colab Notebooks', 'PlantVillage']


In [ ]:
import os

# ✅ Define dataset path
dataset_path = "/content/drive/MyDrive/PlantVillage"  # Update with your dataset path

# ✅ Define train and validation paths
train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "val")

# ✅ Function to count classes and images
def count_classes_and_images(folder_path):
    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return 0, 0  # Return 0 classes and 0 images if folder doesn't exist

    class_folders = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    num_classes = len(class_folders)
    num_images = sum(len(files) for _, _, files in os.walk(folder_path))

    return num_classes, num_images

# ✅ Get class and image counts for train and validation sets
num_train_classes, num_train_images = count_classes_and_images(train_folder)
num_valid_classes, num_valid_images = count_classes_and_images(valid_folder)

# ✅ Print results
print(f"✅ Number of Classes in Training Set: {num_train_classes}")
print(f"✅ Number of Classes in Validation Set: {num_valid_classes}")
print(f"✅ Total Classes: {max(num_train_classes, num_valid_classes)}")  # Assuming same classes in both
print(f"✅ Training Images: {num_train_images}")
print(f"✅ Validation Images: {num_valid_images}")
print(f"✅ Total Images: {num_train_images + num_valid_images}")


✅ Number of Classes in Training Set: 38
✅ Number of Classes in Validation Set: 38
✅ Total Classes: 38
✅ Training Images: 43432
✅ Validation Images: 10861
✅ Total Images: 54293


In [ ]:
import os

# ✅ Define dataset path
dataset_path = "/content/drive/MyDrive/PlantVillage"  # Update with your dataset path
train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "val")

# ✅ Function to count images in each subfolder (class)
def count_images_per_class(folder_path):
    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return {}

    image_counts = {}
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            image_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
            image_counts[class_name] = image_count
    return image_counts

# ✅ Get image counts for train and validation sets
train_image_counts = count_images_per_class(train_folder)
valid_image_counts = count_images_per_class(valid_folder)

# ✅ Print image counts per class
print("✅ Training Set Image Counts per Class:")
for class_name, count in train_image_counts.items():
    print(f"   📁 {class_name}: {count} images")

print("\n✅ Validation Set Image Counts per Class:")
for class_name, count in valid_image_counts.items():
    print(f"   📁 {class_name}: {count} images")

# ✅ Print overall counts
total_train_images = sum(train_image_counts.values())
total_valid_images = sum(valid_image_counts.values())
print(f"\n✅ Total Training Images: {total_train_images}")
print(f"✅ Total Validation Images: {total_valid_images}")
print(f"✅ Total Classes: {max(len(train_image_counts), len(valid_image_counts))}")


✅ Training Set Image Counts per Class:
   📁 Apple___Apple_scab: 492 images
   📁 Apple___Black_rot: 496 images
   📁 Apple___Cedar_apple_rust: 220 images
   📁 Apple___healthy: 1316 images
   📁 Blueberry___healthy: 1202 images
   📁 Cherry_(including_sour)___Powdery_mildew: 842 images
   📁 Cherry_(including_sour)___healthy: 684 images
   📁 Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 410 images
   📁 Corn_(maize)___Common_rust_: 953 images
   📁 Corn_(maize)___Northern_Leaf_Blight: 788 images
   📁 Corn_(maize)___healthy: 929 images
   📁 Grape___Black_rot: 944 images
   📁 Grape___Esca_(Black_Measles): 1107 images
   📁 Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 861 images
   📁 Grape___healthy: 339 images
   📁 Orange___Haunglongbing_(Citrus_greening): 4405 images
   📁 Peach___Bacterial_spot: 1838 images
   📁 Peach___healthy: 288 images
   📁 Pepper,_bell___Bacterial_spot: 797 images
   📁 Pepper,_bell___healthy: 1183 images
   📁 Potato___Early_blight: 800 images
   📁 Potato___Late_blight:

In [ ]:
import os
import cv2
import albumentations as A
from tqdm import tqdm
import numpy as np
from albumentations.pytorch import ToTensorV2

# ✅ Dataset path
dataset_path = "/content/drive/MyDrive/PlantVillage"
train_folder = os.path.join(dataset_path, "train")

# ✅ Count images per class (excluding augmented ones)
def get_class_counts(folder_path):
    class_counts = {}
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            class_counts[label] = len([
                f for f in os.listdir(label_path)
                if os.path.isfile(os.path.join(label_path, f)) and not f.startswith("aug_")
            ])
    return class_counts

# ✅ Augmentation transform (before preprocessing)
augmentation_transform = A.Compose([
    A.HorizontalFlip(p=0.7),
    A.VerticalFlip(p=0.3),
    A.Rotate(limit=30, p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.RandomCrop(height=150, width=150, p=0.4),
])

# ✅ Preprocessing (after augmentation)
preprocess_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# ✅ Augment classes <1000 images to 1000
def augment_to_1000_images(train_folder):
    class_counts = get_class_counts(train_folder)
    total_classes = len(class_counts)

    print("🔁 Starting augmentation for classes with < 1000 images...\n")

    for label, count in tqdm(class_counts.items(), desc="📦 Processing classes", total=total_classes):
        if count >= 1000:
            continue

        label_folder = os.path.join(train_folder, label)
        original_images = [
            f for f in os.listdir(label_folder)
            if os.path.isfile(os.path.join(label_folder, f)) and not f.startswith("aug_")
        ]

        num_to_add = 1000 - count
        print(f"🔄 Augmenting '{label}': {count} → 1000")

        for i in tqdm(range(num_to_add), desc=f"🧪 {label}", leave=False):
            base_image_name = original_images[i % len(original_images)]
            base_image_path = os.path.join(label_folder, base_image_name)

            image = cv2.imread(base_image_path)
            if image is None:
                print(f"⚠️ Skipping unreadable image: {base_image_path}")
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            augmented_image = augmentation_transform(image=image)['image']
            processed = preprocess_transform(image=augmented_image)
            tensor_image = processed['image']

            save_image = tensor_image.permute(1, 2, 0).numpy()
            save_image = np.clip(save_image, 0, 1)
            save_image = (save_image * 255).astype(np.uint8)

            aug_filename = f"aug_fill_{i}_{os.path.basename(base_image_name)}"
            aug_filepath = os.path.join(label_folder, aug_filename)

            success = cv2.imwrite(aug_filepath, cv2.cvtColor(save_image, cv2.COLOR_RGB2BGR))
            if not success:
                print(f"❌ Failed to save: {aug_filepath}")

    print("\n✅ Done: All classes now have at least 1000 original or augmented images.")

# ✅ Run it
augment_to_1000_images(train_folder)


🔁 Starting augmentation for classes with < 1000 images...



📦 Processing classes:   0%|          | 0/38 [00:00<?, ?it/s]

🔄 Augmenting 'Apple___Apple_scab': 492 → 1000



📦 Processing classes:   3%|▎         | 1/38 [00:08<05:25,  8.78s/it]

🔄 Augmenting 'Apple___Black_rot': 496 → 1000


...

🔄 Augmenting 'Tomato___Leaf_Mold': 761 → 1000



📦 Processing classes:  84%|████████▍ | 32/38 [02:49<00:24,  4.13s/it]

🔄 Augmenting 'Tomato___Tomato_mosaic_virus': 299 → 1000



📦 Processing classes: 100%|██████████| 38/38 [03:01<00:00,  4.76s/it]


✅ Done: All classes now have at least 1000 original or augmented images.


In [ ]:
import os

# ✅ Define dataset path
dataset_path = "/content/drive/MyDrive/PlantVillage"  # Update with your dataset path
train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "val")

# ✅ Function to count images in each subfolder (class)
def count_images_per_class(folder_path):
    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return {}

    image_counts = {}
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            image_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
            image_counts[class_name] = image_count
    return image_counts

# ✅ Get image counts for train and validation sets
train_image_counts = count_images_per_class(train_folder)
valid_image_counts = count_images_per_class(valid_folder)

# ✅ Print image counts per class
print("✅ Training Set Image Counts per Class:")
for class_name, count in train_image_counts.items():
    print(f"   📁 {class_name}: {count} images")

print("\n✅ Validation Set Image Counts per Class:")
for class_name, count in valid_image_counts.items():
    print(f"   📁 {class_name}: {count} images")

# ✅ Print overall counts
total_train_images = sum(train_image_counts.values())
total_valid_images = sum(valid_image_counts.values())
print(f"\n✅ Total Training Images: {total_train_images}")
print(f"✅ Total Validation Images: {total_valid_images}")
print(f"✅ Total Classes: {max(len(train_image_counts), len(valid_image_counts))}")


✅ Training Set Image Counts per Class:
   📁 Apple___Apple_scab: 1000 images
   📁 Apple___Black_rot: 1000 images
   📁 Apple___Cedar_apple_rust: 1000 images
   📁 Apple___healthy: 1316 images
   📁 Blueberry___healthy: 1202 images
   📁 Cherry_(including_sour)___Powdery_mildew: 1000 images
   📁 Cherry_(including_sour)___healthy: 1000 images
   📁 Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 1000 images
   📁 Corn_(maize)___Common_rust_: 1000 images
   📁 Corn_(maize)___Northern_Leaf_Blight: 1000 images
   📁 Corn_(maize)___healthy: 1000 images
   📁 Grape___Black_rot: 1000 images
   📁 Grape___Esca_(Black_Measles): 1107 images
   📁 Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1000 images
   📁 Grape___healthy: 1000 images
   📁 Orange___Haunglongbing_(Citrus_greening): 4405 images
   📁 Peach___Bacterial_spot: 1838 images
   📁 Peach___healthy: 1000 images
   📁 Pepper,_bell___Bacterial_spot: 1000 images
   📁 Pepper,_bell___healthy: 1183 images
   📁 Potato___Early_blight: 1000 images
   📁 Potato

In [ ]:
import os
import cv2
import albumentations as A
import numpy as np
from tqdm.notebook import tqdm  # Optimized for Jupyter/Colab
from albumentations.pytorch import ToTensorV2

# ✅ Dataset path
dataset_path = "/content/drive/MyDrive/PlantVillage"
train_folder = os.path.join(dataset_path, "train")

# ✅ Count all images in each class (including augmented)
def get_total_class_counts(folder_path):
    class_counts = {}
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            class_counts[label] = len([
                f for f in os.listdir(label_path)
                if os.path.isfile(os.path.join(label_path, f))
            ])
    return class_counts

# ✅ Augmentation transform
augmentation_transform = A.Compose([
    A.HorizontalFlip(p=0.7),
    A.VerticalFlip(p=0.3),
    A.Rotate(limit=30, p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.RandomCrop(height=150, width=150, p=0.4),
])

# ✅ Preprocessing
preprocess_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# ✅ Augment using only original images until each class has 4500
def augment_classes_to_4500(train_folder):
    class_counts = get_total_class_counts(train_folder)

    for label, count in class_counts.items():
        if count >= 4500:
            continue

        label_folder = os.path.join(train_folder, label)
        all_files = os.listdir(label_folder)
        existing_aug_files = set(all_files)

        # Use only original images (exclude all 'aug_' files)
        original_images = [
            f for f in all_files
            if os.path.isfile(os.path.join(label_folder, f)) and not f.startswith("aug_")
        ]

        if not original_images:
            print(f"⚠️ Skipping '{label}' — no original images found.")
            continue

        needed = 4500 - count
        print(f"🔄 Augmenting '{label}': {count} → 4500 (adding {needed})")

        for i in tqdm(range(needed), desc=f"Augmenting {label}", leave=False):
            base_image_name = original_images[i % len(original_images)]
            base_image_path = os.path.join(label_folder, base_image_name)

            aug_filename = f"aug_4500_{i}_{os.path.basename(base_image_name)}"
            if aug_filename in existing_aug_files:
                continue  # Skip duplicates

            image = cv2.imread(base_image_path)
            if image is None:
                print(f"⚠️ Skipping unreadable image: {base_image_path}")
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            augmented_image = augmentation_transform(image=image)['image']
            processed = preprocess_transform(image=augmented_image)
            tensor_image = processed['image']

            save_image = tensor_image.permute(1, 2, 0).numpy()
            save_image = np.clip(save_image, 0, 1)
            save_image = (save_image * 255).astype(np.uint8)

            aug_filepath = os.path.join(label_folder, aug_filename)
            success = cv2.imwrite(aug_filepath, cv2.cvtColor(save_image, cv2.COLOR_RGB2BGR))

            if not success:
                print(f"❌ Failed to save: {aug_filepath}")

    print("✅ All classes now have exactly 4500 images using only originals for augmentation.")

# ✅ Run it
augment_classes_to_4500(train_folder)


🔄 Augmenting 'Peach___healthy': 4256 → 4500 (adding 244)


Augmenting Peach___healthy:   0%|          | 0/244 [00:00<?, ?it/s]

✅ All classes now have exactly 4500 images using only originals for augmentation.


In [ ]:
import os

# ✅ Define dataset path
dataset_path = "/content/drive/MyDrive/PlantVillage"  # Update with your dataset path
train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "val")

# ✅ Function to count images in each subfolder (class)
def count_images_per_class(folder_path):
    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return {}

    image_counts = {}
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            image_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
            image_counts[class_name] = image_count
    return image_counts

# ✅ Get image counts for train and validation sets
train_image_counts = count_images_per_class(train_folder)
valid_image_counts = count_images_per_class(valid_folder)

# ✅ Print image counts per class
print("✅ Training Set Image Counts per Class:")
for class_name, count in train_image_counts.items():
    print(f"   📁 {class_name}: {count} images")

print("\n✅ Validation Set Image Counts per Class:")
for class_name, count in valid_image_counts.items():
    print(f"   📁 {class_name}: {count} images")

# ✅ Print overall counts
total_train_images = sum(train_image_counts.values())
total_valid_images = sum(valid_image_counts.values())
print(f"\n✅ Total Training Images: {total_train_images}")
print(f"✅ Total Validation Images: {total_valid_images}")
print(f"✅ Total Classes: {max(len(train_image_counts), len(valid_image_counts))}")


✅ Training Set Image Counts per Class:
   📁 Apple___Apple_scab: 4500 images
   📁 Apple___Black_rot: 4500 images
   📁 Apple___Cedar_apple_rust: 4500 images
   📁 Apple___healthy: 4500 images
   📁 Blueberry___healthy: 4500 images
   📁 Cherry_(including_sour)___Powdery_mildew: 4500 images
   📁 Cherry_(including_sour)___healthy: 4500 images
   📁 Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 4500 images
   📁 Corn_(maize)___Common_rust_: 4500 images
   📁 Corn_(maize)___Northern_Leaf_Blight: 4500 images
   📁 Corn_(maize)___healthy: 4500 images
   📁 Grape___Black_rot: 4500 images
   📁 Grape___Esca_(Black_Measles): 4500 images
   📁 Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 4500 images
   📁 Grape___healthy: 4500 images
   📁 Orange___Haunglongbing_(Citrus_greening): 4500 images
   📁 Peach___Bacterial_spot: 4500 images
   📁 Peach___healthy: 4256 images
   📁 Pepper,_bell___Bacterial_spot: 4500 images
   📁 Pepper,_bell___healthy: 4500 images
   📁 Potato___Early_blight: 4500 images
   📁 Potato

In [ ]:
!nvidia-smi

Sat Apr 26 02:57:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import os
import time
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import UnidentifiedImageError

# ✅ Device & AMP Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()

print(f"Using device: {device}")

# ✅ Dataset Paths
dataset_path = "/content/drive/MyDrive/PlantVillage"
train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "val")

# ✅ Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Custom Dataset to Handle Corrupt Images
class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except (UnidentifiedImageError, OSError, ValueError):
            print(f"⚠️ Skipping corrupt: {self.samples[index][0]}")
            dummy = torch.zeros(3, 128, 128)
            return dummy, -1

# ✅ Load Datasets with More Workers
train_dataset = SafeImageFolder(train_folder, transform)
val_dataset = SafeImageFolder(valid_folder, transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# ✅ Number of Classes
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")

# ✅ EfficientNet-B0 Model
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
model.to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Checkpoint Paths
checkpoint_path = "/content/drive/MyDrive/efficientnet_checkpoint.pth"
final_model_path = "/content/drive/MyDrive/efficientnet_final.pth"

# ✅ Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    best_val_acc = 0.0
    start_epoch = 0

    # Resume from Checkpoint
    if os.path.exists(checkpoint_path):
        print("🔄 Loading checkpoint...")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_acc = checkpoint['best_val_acc']
        print(f"✅ Resumed from epoch {start_epoch} (best acc: {best_val_acc:.2f}%)")

    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_loss = 0.0
        correct, total = 0, 0
        epoch_start = time.time()

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            mask = labels != -1
            if not mask.any():
                continue
            images, labels = images[mask].to(device), labels[mask].to(device)

            optimizer.zero_grad()
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total if total > 0 else 0
        print(f"✅ Epoch {epoch+1}, Loss: {total_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

        # ✅ Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                mask = labels != -1
                if not mask.any():
                    continue
                images, labels = images[mask].to(device), labels[mask].to(device)

                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)

        val_acc = 100 * correct / total if total > 0 else 0
        print(f"📈 Validation Accuracy: {val_acc:.2f}%")
        print(f"⏱️ Epoch Time: {time.time() - epoch_start:.2f} sec\n")

        # ✅ Save Checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': max(best_val_acc, val_acc)
        }, checkpoint_path)
        print(f"💾 Checkpoint saved (Epoch {epoch+1})")

        if val_acc > best_val_acc:
            best_val_acc = val_acc

    print(f"🎯 Best Validation Accuracy: {best_val_acc:.2f}%")
    torch.save(model.state_dict(), final_model_path)
    print(f"🎉 Final model saved to: {final_model_path}")

# ✅ Run the Training
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)


<ipython-input-16-a9c4ccda8947>:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Using device: cuda
Number of classes: 38


Epoch 1/5: 100%|██████████| 2669/2669 [2:53:01<00:00,  3.89s/it]

✅ Epoch 1, Loss: 1044.3447, Train Accuracy: 87.84%


📈 Validation Accuracy: 98.58%
⏱️ Epoch Time: 12253.80 sec

💾 Checkpoint saved (Epoch 1)


...

✅ Epoch 4, Loss: 378.8810, Train Accuracy: 95.40%


📈 Validation Accuracy: 98.39%
⏱️ Epoch Time: 353.06 sec

💾 Checkpoint saved (Epoch 4)


Epoch 5/5: 100%|██████████| 2669/2669 [05:29<00:00,  8.09it/s]

✅ Epoch 5, Loss: 327.1543, Train Accuracy: 96.01%


📈 Validation Accuracy: 99.29%
⏱️ Epoch Time: 351.79 sec

💾 Checkpoint saved (Epoch 5)
🎯 Best Validation Accuracy: 99.29%
🎉 Final model saved to: /content/drive/MyDrive/efficientnet_final.pth


In [ ]:
import os
import time
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image, UnidentifiedImageError  # Importing Image properly
import numpy as np  # Importing numpy for array conversion


In [ ]:
import os
import time
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import UnidentifiedImageError

# ✅ Device & AMP Setup (A100 or any CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()
print(f"🚀 Using device: {device}")

# ✅ Dataset Paths
dataset_path = "/content/drive/MyDrive/PlantVillage"
train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "val")

# ✅ Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Custom Dataset to Handle Corrupt Images
class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except (UnidentifiedImageError, OSError, ValueError):
            print(f"⚠️ Skipping corrupt image: {self.samples[index][0]}")
            dummy = torch.zeros(3, 128, 128)
            return dummy, -1

# ✅ Load Datasets
train_dataset = SafeImageFolder(train_folder, transform)
val_dataset = SafeImageFolder(valid_folder, transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# ✅ Number of Classes
num_classes = len(train_dataset.classes)
print(f"📊 Number of classes: {num_classes}")

# ✅ EfficientNet-B0 Model
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
model.to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Checkpoint Paths
checkpoint_path = "/content/drive/MyDrive/efficientnet_checkpoint.pth"
final_model_path = "/content/drive/MyDrive/efficientnet_final.pth"

# ✅ Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, total_epochs=20):
    start_epoch = 0
    best_val_acc = 0.0

    # ✅ Resume from checkpoint if exists
    if os.path.exists(checkpoint_path):
        print("🔄 Loading checkpoint...")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_acc = checkpoint['best_val_acc']
        print(f"✅ Resumed from epoch {start_epoch} (Best Val Acc: {best_val_acc:.2f}%)")

    for epoch in range(start_epoch, total_epochs):
        model.train()
        total_loss = 0.0
        correct, total = 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{total_epochs}"):
            mask = labels != -1
            if not mask.any():
                continue
            images, labels = images[mask].to(device), labels[mask].to(device)

            optimizer.zero_grad()
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total if total > 0 else 0

        # ✅ Validation
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                mask = labels != -1
                if not mask.any():
                    continue
                images, labels = images[mask].to(device), labels[mask].to(device)

                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
                    val_correct += (predicted == labels).sum().item()
                    val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total if val_total > 0 else 0

        # ✅ Print both accuracies
        print(f"📊 Epoch {epoch+1} - Train Accuracy: {train_acc:.2f}% | 📈 Val Accuracy: {val_acc:.2f}%")

        # ✅ Save Checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': max(best_val_acc, val_acc)
        }, checkpoint_path)

        if val_acc > best_val_acc:
            best_val_acc = val_acc

    torch.save(model.state_dict(), final_model_path)
    print(f"🎯 Best Validation Accuracy: {best_val_acc:.2f}%")
    print(f"🎉 Final model saved to: {final_model_path}")

# ✅ Start Training (e.g., continue training for total 20 epochs)
train_model(model, train_loader, val_loader, criterion, optimizer, total_epochs=20)


<ipython-input-41-1f4c0497f9ea>:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


🚀 Using device: cuda
📊 Number of classes: 38
🔄 Loading checkpoint...
✅ Resumed from epoch 5 (Best Val Acc: 99.29%)


Epoch 6/20: 100%|██████████| 2669/2669 [05:59<00:00,  7.43it/s]


📊 Epoch 6 - Train Accuracy: 96.62% | 📈 Val Accuracy: 98.97%


Epoch 7/20: 100%|██████████| 2669/2669 [05:30<00:00,  8.07it/s]


...

📊 Epoch 18 - Train Accuracy: 98.66% | 📈 Val Accuracy: 99.46%


Epoch 19/20: 100%|██████████| 2669/2669 [05:31<00:00,  8.04it/s]


📊 Epoch 19 - Train Accuracy: 98.67% | 📈 Val Accuracy: 99.21%


Epoch 20/20: 100%|██████████| 2669/2669 [05:31<00:00,  8.06it/s]


📊 Epoch 20 - Train Accuracy: 98.78% | 📈 Val Accuracy: 99.06%
🎯 Best Validation Accuracy: 99.46%
🎉 Final model saved to: /content/drive/MyDrive/efficientnet_final.pth


In [ ]:
# Print model layers and check dropout layers
print(model)


EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2